In [44]:
import pandas as pd
import unidecode
import re

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import spacy

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bapti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
gmail = pd.read_csv("data/gmail_oscar.CSV")

zimbra = pd.read_csv("data/zimbra_oscar.CSV")

In [46]:
datasets = [gmail, zimbra]

In [47]:
def text_cleaning(df):

    i = 0
    for corps in df['Corps']:
        text_file = open(r'corps.txt', 'w',  encoding="utf-8")
        text_file.write(corps)
        text_file.close()

        
        final_text_file = open(r'final_corps.txt', 'w',  encoding="utf-8")
        reading_text_file = open(r'corps.txt', 'r',  encoding="utf-8")
        for line in reading_text_file:
            if "https" not in line:
                final_text_file.write(line)
            
        final_text_file.close()
        reading_text_file.close()

        with open('final_corps.txt', 'r', encoding="utf-8") as file:
            data = file.read()
        
    
        data = unidecode.unidecode(data)
        data = data.lower()
        data = re.sub(r"""[.,"'-:!;]""", '', data)
        data = re.sub('\s+', ' ', data)
        data = data.split(' ')


        # print("avant de retirer les stopwords : ", len(data), " mots")
        stop_words = stopwords.words('french')
        data = [word for word in data if word not in stopwords.words('french') and len(word)>1]
        # print("Après avoir retiré les stopwords : ", len(data), " mots")

        # Lemmatization
        nlp_fr = spacy.load('fr_core_news_md')
        nlp_en = spacy.load('en_core_web_md')
        j = 0
        # On regarde chaque mot dans data, qui est le corps numéro i
        # Chaque mot a le numéro j
        for mot in data:
            # on va lemmatizer en anglais et en français
            doc_fr = nlp_fr(mot)
            doc_en = nlp_en(mot)
            for token in doc_fr:
                token_or = token.text
                token_fr = token.lemma_
            for token in doc_en:
                token_en = token.lemma_
            
            # on vérifie quelle langue a été lemmatizée, ou aucune
            if token_or != token_fr:
                lemma = token_fr
            elif token_or != token_en:
                lemma = token_en
            else:
                lemma = token_or
            
            # on change le mot numéro j dans data
            data[j] = lemma
            j+=1


        df["Corps"][i] = data

        i = i+1
    return df

In [50]:
print(len(gmail),len(zimbra))

10 120


# ATTENTION

Zimbra a 12 fois plus de mails que gmail, la lemmatization prend donc 12 fois plus de temps !

33min pour moi :O

Voir comment modifier pour ne pas lemmatizer tous les mots dans les 2 langues -> diviser le temps de traitement par 2

Pas obligé de lancer la boucle for qui suit pour tester -> tester que sur gmail

## Idée pour réduire le temps

traiter l'objet et le corps dans une même fonction

tester les 2 langues pour l'objet, et vérifier quelle lemmatization a modifié le + de mots -> langue de l'objet

une fois qu'on connait la langue de l'objet, on peut lemmatizer le corps avec la bonne langue

Ne marche pas que si l'objet et le corps ne sont pas dans la même langue ou si le corps contient les 2 langues

In [48]:
for df in datasets:
    df = text_cleaning(df)

C:\Users\bapti\AppData\Local\Temp\ipykernel_10080\1008173592.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Corps"][i] = data


In [53]:
gmail['Corps']

0    [octobre, bientot, oscar, voici, facture, cour...
1    [vouloir, repondre, audessus, ce, ligne, repon...
2    [octobre, oscar, merci, davoir, utilise, uber,...
3    [decouvrer, vite, selection, jusqua, demain, s...
4    [annoncer, correspondre, recherche, location, ...
5    [annoncer, correspondre, recherche, location, ...
6    [erinette, repondu, avis, merci, davoir, publi...
7    [rencontrer, problme, toucher, paiement, premi...
8    [cher, client, cher, client, informer, recepti...
9    [solde, bas, bonjour, oscar, solde, compte, ca...
Name: Corps, dtype: object